# Use Folium to Display Maps

In [2]:
import folium
map_osm = folium.Map(location=[45.5236, -122.6750])
map_osm.save('osm.html')

# Bike Routes in Dublin

In [1]:
import pandas as pd
import folium
location = "https://data.smartdublin.ie/dataset/33ec9fe2-4957-4e9a-ab55-c5e917c7a9ab/resource/2dec86ed-76ed-47a3-ae28-646db5c5b965/download/dublin.csv"
bike_station_locations = pd.read_csv(location)
bike_station_locations = bike_station_locations[["Latitude", "Longitude", "Name"]]
map = folium.Map(location=[bike_station_locations.Latitude.mean(), bike_station_locations.Longitude.mean()], zoom_start=14, control_scale=True)
for index, location_info in bike_station_locations.iterrows():
    folium.Marker([location_info["Latitude"], location_info["Longitude"]], popup=location_info["Name"]).add_to(map)

# save to html file
map.save("index.html")

In [2]:
import pandas as pd
import folium
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()



Successfully saved authorization token.


In [17]:


# Define a method for displaying Earth Engine image tiles to folium map.



styleParams = {
  'fillColor': 'b5ffb4',
  'color': '00909F',
  'width': 3.0,
};


# LSIB Large Scale International Boundaries
worldcountries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017');
filterCountry = ee.Filter.eq('country_na', 'United Kingdom');
country = worldcountries.filter(filterCountry);

# Define the visualization parameters.


landsat = ee.ImageCollection(
    'LANDSAT/LC08/C01/T1').filterDate('2016-01-01', '2017-01-01').filterBounds(country)
composite = ee.Algorithms.Landsat.simpleComposite(**{
    'collection': landsat,
    'asFloat': True
})
rgbVis = {'bands': ['B3', 'B4'], 'min': 0, 'max': 0.3}
nirVis = {'bands': ['B5', 'B4'], 'min': 0, 'max': [0.5, 0.3]}




def add_ee_layer(self, eeImageObject, visParams, name):
  mapID = ee.Image(eeImageObject).getMapId(visParams)
  folium.raster_layers.TileLayer(
    tiles = "https://earthengine.googleapis.com/map/"+mapID['mapid']+
      "/{z}/{x}/{y}?token="+mapID['token'],
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Fetch an elevation model.
dem = ee.Image('USGS/SRTMGL1_003')
name = ee.String('FIDO')

# Set visualization parameters.
visParams = {'min':0, 'max':3000}

# Create a folium map object.
# Center the map and display the image.


myMap = folium.Map(location=[52.1307, 3.7837], zoom_start=10, height=500)

# Add the elevation model to the map object.
#myMap.add_ee_layer(dem, visParams, 'DEM')
myMap.add_ee_layer(composite, rgbVis, 'RGB')
myMap.add_ee_layer(composite, nirVis, 'false Color')


# Add a layer control panel to the map.
myMap.add_child(folium.LayerControl())

# Display the map.
myMap.save("myMap.html")

EEException: Landsat.simpleComposite: No images in collection.